# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        fail = True
        while fail:
            bgnDex   = randrange(0,Nchunks) * chunkSize
            moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
            sendDex  = randrange(0,Ntracks-1)
            try:
                response = spot.playlist_reorder_items(
                    playlist_ID, 
                    bgnDex, 
                    sendDex, 
                    range_length = chunkSize
                )
                if verbose:
                    print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
                fail = False
            except spotipy.SpotifyException as ex:
                if verbose:
                    print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'study15' : "6SqlfurCBP7eeMOojaDNtS",
    'study16' : "5TtKaKCouyJp7Hhtu4YlYm",
    'study17' : "5qX1Tq3IQ74iSgrlMfRhty",
    'study18' : "1bwXMYoRgEFWebWZ9ZPBqs",
    'study19' : "3ackEOD2vox5Oc1vjeALKJ", 
    'study20' : "3Cmpe4nGzOQDefowW49pBS",
    'PianoJz' : "2IqcxvGty13ewL583z9Kdn",
    'cringe1' : "2AAUYlKM1nXKHkZUpUSFbv",
    'zk_Over' : "1diwH003mkfg3cLejTuKTN",
}

complete_reorder_entire_playlist( playlist['zk_Over'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 500-600 --> 304:
	{'snapshot_id': 'AAADomHxQBTIGvVaVfKQOiOa48VbOLPd'}
Iteration 2, 300-400 --> 479:
	{'snapshot_id': 'AAADo3FAagfrzoDwoFLH7N6La4D25dk0'}
Iteration 3, 800-900 --> 376:
	{'snapshot_id': 'AAADpG5t6pWuapNcXTreGYZvY1I2XOQC'}
Iteration 4, 0-100 --> 268:
	{'snapshot_id': 'AAADpbtKo+6sTQR4+wA2AihD7zObbY1g'}
Iteration 5, 600-700 --> 788:
	{'snapshot_id': 'AAADpsD2XwXgPVfTk6aEFIfq6s1Yyo39'}


HTTP Error for PUT to https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 6, 100-200 --> 892:
	{'snapshot_id': 'AAADp2UZLwIjDexdfZG2Ax4EmhjU+7mp'}
ERROR, 1000-1020 --> 460:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 7, 200-300 --> 813:
	{'snapshot_id': 'AAADqHKeaAviCS3KgVVkbTJQdggOgvKo'}
Iteration 8, 500-600 --> 206:
	{'snapshot_id': 'AAADqcYbbqM6BfCNiEPIjHq5estTa2yQ'}
Iteration 9, 400-500 --> 824:
	{'snapshot_id': 'AAADqv9SJ8PVzJFHOixtvi6AXGEiMfzS'}
Iteration 10, 600-700 --> 158:
	{'snapshot_id': 'AAADqzlaUbTnvAQeFGBzks04o1t6Nz6T'}

Phase 2: Chunks of 10

Iteration 1, 410-420 --> 961:
	{'snapshot_id': 'AAADrNlrCKiR58d805dagOOQ0sW/kog/'}
Iteration 2, 660-670 --> 889:
	{'snapshot_id': 'AAADrQJ9XU681OPVhb4NhEhFCp7hYREG'}
Iteration 3, 770-780 --> 91:
	{'snapshot_id': 'AAADrv8hmWvNxgAoaaeQQ5IUrmCzcTKE'}
Iteration 4, 570-580 --> 920:
	{'snapshot_id': 'AAADrxTPAoAASQpN9JHSXAzYspIWyrOH'}
Iteration 5, 490-500 --> 891:
	{'snapsho

HTTP Error for PUT to https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 38, 645-650 --> 20:
	{'snapshot_id': 'AAAEN7jMrwQ8IH4Xwfz/RSJIPhu/TyKq'}
ERROR, 1020-1020 --> 112:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 39, 355-360 --> 557:
	{'snapshot_id': 'AAAEOEcWsD5jsR1FOvpnbB6tU9mv7h+i'}
Iteration 40, 980-985 --> 94:
	{'snapshot_id': 'AAAEOZ/dBxBePARuqDcR21dwvVWELK+x'}
Iteration 41, 955-960 --> 776:
	{'snapshot_id': 'AAAEOsGJE3OvEbcWgTKEA+JeNHTk89WQ'}
Iteration 42, 675-680 --> 913:
	{'snapshot_id': 'AAAEO0/fmugwh+krMeQrXu2k+kMMPeUj'}
Iteration 43, 315-320 --> 353:
	{'snapshot_id': 'AAAEPOeBO5et7cUoCkwN4ZY5b99LZmTs'}
Iteration 44, 395-400 --> 978:
	{'snapshot_id': 'AAAEPafm99a2dg2fM8C30s5ivGBYcbCb'}
Iteration 45, 290-295 --> 71:
	{'snapshot_id': 'AAAEPsWWYTaKML3wI1WUQB0XV1IU5TzM'}
Iteration 46, 580-585 --> 943:
	{'snapshot_id': 'AAAEP06fkJQ3+qFOzrNYZqyuNthnD27T'}
Iteration 47, 120-125 --> 611:
	{'snapshot_id': 'AAAEQAJ+x

HTTP Error for PUT to https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 77, 295-300 --> 564:
	{'snapshot_id': 'AAAEXjT0Obn+c0FR9lUTaslErk16XF1i'}
ERROR, 1020-1020 --> 323:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 78, 50-55 --> 449:
	{'snapshot_id': 'AAAEX7rBe4ExylU69eV7QK173MjV3tTL'}
Iteration 79, 275-280 --> 774:
	{'snapshot_id': 'AAAEYCkiMTKz1CrnEwH1X0NEDVJHLR6s'}
Iteration 80, 820-825 --> 686:
	{'snapshot_id': 'AAAEYaPd9GpB5nxtgNItbzlbLIgx74no'}
Iteration 81, 580-585 --> 903:
	{'snapshot_id': 'AAAEYn5ps5zbtb5Pkb/5uBp2DL4PQvRE'}
Iteration 82, 965-970 --> 767:
	{'snapshot_id': 'AAAEY5zRtgscZX+lEvOHdsiv5ewmWvp7'}
Iteration 83, 250-255 --> 610:
	{'snapshot_id': 'AAAEZF1SSpbuplJX3bo/KxSlw8Id0Ie9'}
Iteration 84, 170-175 --> 919:
	{'snapshot_id': 'AAAEZbrKvGreU4svU/JJbBEZLVATdy18'}
Iteration 85, 175-180 --> 854:
	{'snapshot_id': 'AAAEZj8R7HkarQ0v3OlmmI367X0MXpqx'}
Iteration 86, 160-165 --> 932:
	{'snapshot_id': 'AAAEZ7J9